# Recurrent neural network(RNN) 


Recurrent neural network(RNN)은 Text, 음성, 시계열 데이터등 순차적인(Sequential) 데이터들에 대한 분석을 할때 주로 사용하는 알고리즘입니다. <br>
본 문서에서는 지난시간에 배웠던 RNN을 활용해 '임의의 순서'를 갖는 문자열 데이터로부터<br> 'I LOVE YOU'라는 순서를 갖는 문자열을 출력하는 RNN을 학습시켜 보겠습니다.<br>

본 예제에서는<br>
Input: UOYEVOLI -> Output: ILOVEYOU <br>
되도록 학습 시킬 것 입니다.<br><br>
이렇게 input과 output이 모두 Sequence인 모델을 <br>
- sequence-to-sequence(seq2seq) 
- many-to-many model
- encoder, decoder model

이라고 부릅니다. 대표적인 예로는 Neural Machine Translation(NMT)가 있습니다.

In [1]:
#-*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint #데이터를 보기 좋게 출력해주는 모듈

자연어 처리에서는 보통 데이터를 다룰때 <br>
1.단어사전) '텍스트데이터':'인덱스' <br>
2.인덱스를 단어로변환) '인덱스':'텍스트데이터'<br>
의 형태로 구축해서 사용합니다. <br>
데이터를 처리할때 '텍스트데이터' 형태로 그대로 쓰기보다 '인덱스'를 사용해서 처리하는 것이 통상적이므로 여기서도 같은 방법을 사용하겠습니다.

In [2]:
text_data = ['I','L','O','V','E','Y','O','U']


# 단어사전 구축
vocab = {}
index = 0
for text in text_data:
    if text not in vocab:
        vocab[text] = index
        index = index + 1

# 인덱스->단어 사전 구축
index2Char = {}
for text, index in vocab.items():
    index2Char[index] = text

print(vocab)
print(index2Char)

# one hot representation 차원 계산
one_hot_dimension = len(vocab)
print("one_hot_dimension:", one_hot_dimension)

# 넣어줄 데이터를 np.float32 타입으로 변환해줍니다. (지정하지 않아서 타입이 안맞을 경우 에러 발생)
# 각 문자를 벡터로 나타내기 위해 one-hot representation 형태를 사용합니다.
one_hot_embedding_matrix = np.array([[1, 0, 0, 0, 0, 0, 0],     # I
                                       [0, 1, 0, 0, 0, 0, 0],  # L
                                       [0, 0, 1, 0, 0, 0, 0],  # O
                                       [0, 0, 0, 1, 0, 0, 0],  # V
                                       [0, 0, 0, 0, 1, 0, 0],  # E
                                       [0, 0, 0, 0, 0, 1, 0],  # Y
                                       [0, 0, 0, 0, 0, 0, 1]], dtype=np.float32) # U

print(one_hot_embedding_matrix)

{'O': 2, 'L': 1, 'V': 3, 'U': 6, 'I': 0, 'E': 4, 'Y': 5}
{0: 'I', 1: 'L', 2: 'O', 3: 'V', 4: 'E', 5: 'Y', 6: 'U'}
one_hot_dimension: 7
[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]


In [3]:
y_data = [[0, 1, 2, 3, 4, 5, 2, 6]]    # Target은 ILOVEYOU 입니다.
y_string = [index2Char[y] for y in y_data[0]]
print("y_data", y_data)
print("y_string", y_string)

# I LOVE YOU를 거꾸로 뒤집은 "UOY EVOL I"을 임의의 문자열로 선택했습니다.
x_input_char = ['U','O','Y','E','V','O','L','I']

# 문자열을 index로 바꾸겠습니다.
x_input_index = []

for x in x_input_char:    
    x_input_index.append(vocab[x])  #  (문자->index) 로 변환

print("x_input_index", x_input_index)    
x_input_string = [index2Char[x] for x in x_input_index]
print("x_input_string", x_input_string)

# index를 이용하여 one_hot_vector 모양으로 바꾸겠습니다.
x_input_one_hot = []
for x in x_input_index:
    x_input_one_hot.append(one_hot_embedding_matrix[x])
x_input_one_hot = np.array([x_input_one_hot]) #Tensorflow에서는 첫차원이 Batch size이므로 차원을 한칸 밀어줍니다.
print("x_input_one_hot")
print(x_input_one_hot) # Batch, 문장내 문자수, 문자 차원수


y_data [[0, 1, 2, 3, 4, 5, 2, 6]]
y_string ['I', 'L', 'O', 'V', 'E', 'Y', 'O', 'U']
x_input_index [6, 2, 5, 4, 3, 2, 1, 0]
x_input_string ['U', 'O', 'Y', 'E', 'V', 'O', 'L', 'I']
x_input_one_hot
[[[ 0.  0.  0.  0.  0.  0.  1.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.]
  [ 0.  0.  0.  0.  1.  0.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.  0.  0.]]]


In [4]:
# parameters

sequence_length = 8 #len(x_input_char)
input_dimension = 7 #one_hot_dimension
num_classes = 7 # 문자수
hidden_size = 2 # RNN의 hidden layer 차원
num_layers = 2 # Multi layer RNN의 layer 개수
batch_size = 1 # 현재는 문장 1개로 하기 때문에 1
learning_rate = 0.01 # Adam optimizer의 Learning rate

In [5]:
X_input = tf.placeholder(tf.float32, [None, sequence_length, input_dimension]) #Padding까지 고려된 크기
Y = tf.placeholder(tf.int32, [None, sequence_length]) # RNN의 output은 character에 대한 index (Class 혹은 Label)
seq_length = tf.placeholder(tf.int32) #나중에 가변적인 값을 넣을 때 사용 ex) [['really','good'],['good', 'pad']] 의 경우 Seq_length는 [2,1]

In [6]:
sess = tf.InteractiveSession()

### Multi-Bidirectional RNN
지난 시간에 배운 Multi-Bidirectional RNN을 적용해보겠습니다

In [9]:
def Multi_Bi_RNN(x_input, sequence_length, hidden_size, num_layers, rnn_type='rnn'):
    with tf.variable_scope('multi_bidirectional_rnn') as scope:
        
        
        cell_fw = None
        cell_bw = None
        if(rnn_type == 'rnn'):
            cell_fw = [tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) for i in range(num_layers)]
            cell_bw = [tf.contrib.rnn.BasicRNNCell(num_units=hidden_size) for i in range(num_layers)]
            
        elif(rnn_type == 'lstm'):
            cell_fw = [tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) for i in range(num_layers)]
            cell_bw = [tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) for i in range(num_layers)]
            #state_is_tuple = True로 하면 Cell State와 Hidden State의 값을 tuple형태로 분리해서 보여줍니다.
        cell_fw = tf.contrib.rnn.MultiRNNCell(cells=cell_fw, state_is_tuple=True)
        cell_bw = tf.contrib.rnn.MultiRNNCell(cells=cell_bw, state_is_tuple=True)
            
        outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_input, 
                                                          sequence_length=sequence_length,
                                                          dtype=tf.float32)
        return outputs, states

In [10]:
outputs, states = Multi_Bi_RNN(X_input, seq_length, hidden_size, num_layers, rnn_type='lstm')

### Fully Connected Layer
RNN에서 나온 값들을 Fully Connected Layer의 Input 값으로 사용합니다.

In [11]:
# x_for_fc_0 = tf.reshape(outputs[0], [-1, hidden_size])
# x_for_fc_1 = tf.reshape(outputs[1], [-1, hidden_size])
# x_for_fc = tf.concat(x_for_fc_0, x_for_fc_1, axis = 1)

x_for_fc = tf.reshape(outputs, [-1, hidden_size * 2]) # Bidirectional 이기 때문에 Output이 2개 나오기 때문에 hidden_size * 2

outputs = tf.contrib.layers.fully_connected(
    inputs=x_for_fc, num_outputs=num_classes, activation_fn=None, weights_initializer=tf.contrib.layers.xavier_initializer()) # Xavier Glorot and Yoshua Bengio (2010): Understanding the difficulty of training deep feedforward neural networks. 
#print(outputs) #Tensor("fully_connected/BiasAdd:0", shape=(?, 7), dtype=float32)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes]) #TODO Seq_length
#print(outputs) #Tensor("Reshape_2:0", shape=(1, 8, 7), dtype=float32)

### Seq2Seq Loss 계산 및 Train op
Time Step 마다 나오는 output과 거기에 대응되는 Target Y의 loss를 계산해줍니다.<br>
Tensorflow에서는 이를 위한 API인 tf.contrib.seq2seq.sequence_loss를 제공해줍니다.<br>
weights는 주로 padding이 있을때 masking 용도로 사용합니다.

In [12]:
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [13]:
prediction = tf.argmax(outputs, axis=2) # 0: Batch_size, 1: Sequence_length, 2: Num_classes

In [ ]:
sess.run(tf.global_variables_initializer())

## Training & Evaluation
정의한 Operation을 session에 넣고 training을 시작합니다.<br>
모델이 예측하는 문자열의 순서를 확인합니다.<br>

In [ ]:
for i in range(200):
        l, _ = sess.run([loss, train], feed_dict={X_input: x_input_one_hot, Y: y_data, seq_length:[8]}) # seq_length:[sequence_length] (Batch_size, Seq)
        result = sess.run(prediction, feed_dict={X_input: x_input_one_hot, seq_length:[8]})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [index2Char[c] for c in np.squeeze(result)]
        print("RNN 모델의 예측 결과: ", ''.join(result_str))
print("End:)")

0 loss: 1.94265 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
1 loss: 1.93258 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
2 loss: 1.92225 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
3 loss: 1.91143 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
4 loss: 1.89994 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
5 loss: 1.88767 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
6 loss: 1.87458 prediction:  [[2 2 2 2 4 4 4 2]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEO
7 loss: 1.8607 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
8 loss: 1.84605 prediction:  [[2 2 2 2 4 4 4 4]] true Y:  [[0, 1, 2, 3, 4, 5, 2, 6]]
RNN 모델의 예측 결과:  OOOOEEEE
9 loss: 1.8